In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 7.3 MB/s eta 0:00:00


In [3]:
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.8 MB/s eta 0:00:00


In [4]:
# import basic libraries
import pandas as pd
import numpy as np
import os
import re
import random
import string
import nltk
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk import word_tokenize
import torch

nltk.download('stopwords')
nltk.download("punkt")
nltk.download('wordnet')
nltk.download('omw-1.4')
warnings.filterwarnings(action = 'ignore')


from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import cluster 
from sklearn import metrics
from sklearn.decomposition import PCA 
from sklearn.manifold import TSNE 
from gensim.models import Word2Vec

SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [5]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from transformers import pipeline


In [6]:
#Training Dataset
dataset = pd.read_csv('BBC_News.csv')


In [7]:
dataset.head()

,Category,Text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [8]:
# delete the duplicate article
dataset=dataset.drop_duplicates(subset={"Category","Text"}, keep='first', inplace=False)
dataset.shape 

(2126, 2)

In [9]:
df = dataset.iloc[:2500]
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2126 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  2126 non-null   object
 1   Text      2126 non-null   object
dtypes: object(2)
memory usage: 49.8+ KB


(2126, 2)

In [10]:
df['Label'] = df['Category']
df['Label'].replace(['business','entertainment','tech','politics','sport'],[4,2,0,1,3],inplace=True)

In [11]:
df2 = df[['Text', 'Label']]
df2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2126 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    2126 non-null   object
 1   Label   2126 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 49.8+ KB


In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [55]:
feature_extraction_pipe = pipeline('feature-extraction', model='roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [56]:
pipelineVecs = []
for index, row in df2.iterrows():
  output = feature_extraction_pipe(row["Text"][0][0])
  pipelineVecs.append(np.array(output[0][0]))

In [57]:
df2['pipelineVecs'] = pipelineVecs

In [58]:
train_split = int(len(df2) * 0.8)
tmp_train = df2.iloc[:train_split,:]
test_data = df2.iloc[train_split:,:]

In [59]:
validation_split = int(train_split * 0.8)
train_data = tmp_train.iloc[:validation_split,:]
validation_data = tmp_train.iloc[validation_split:,:]

In [60]:
train_labels = train_data['Label']
validation_labels = validation_data['Label']
test_labels = test_data['Label']

In [61]:
trainingVector = train_data['pipelineVecs']
validationVector = validation_data['pipelineVecs']

In [62]:
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression(random_state=42, max_iter=10000)
logistic_regression.fit(trainingVector.tolist(), train_labels)
validation_labels_pred = logistic_regression.predict(validationVector.tolist())

In [63]:
from sklearn.metrics import classification_report
print(classification_report(logistic_regression.predict(validationVector.tolist()), validation_labels,digits=3))

              precision    recall  f1-score   support

           0      0.000     0.000     0.000         0
           1      0.288     0.420     0.341        50
           2      0.000     0.000     0.000         0
           3      0.595     0.342     0.435       146
           4      0.493     0.229     0.313       144

    accuracy                          0.306       340
   macro avg      0.275     0.198     0.218       340
weighted avg      0.507     0.306     0.369       340



In [19]:
import numpy as np
import sys 

from transformers import pipeline
from transformers import RobertaTokenizer, TFRobertaModel

In [20]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [21]:
inputs = []
masks = []

for index, row in validation_data.iterrows():
  encoding = tokenizer(row["Text"],padding=True, truncation=True)
  inputs.append(encoding['input_ids'])
  masks.append(encoding['attention_mask'])


validation_data["input_ids"] = inputs
validation_data["attention_mask"] = masks

inputs = []
masks = []

for index, row in train_data.iterrows():
  encoding = tokenizer(row["Text"],padding=True, truncation=True)
  inputs.append(encoding['input_ids'])
  masks.append(encoding['attention_mask'])


train_data["input_ids"] = inputs
train_data["attention_mask"] = masks




In [22]:
!pip install dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.2 MB/s eta 0:00:00


In [23]:
from datasets import Dataset
ids = []
masks = []
lbls = []
for index, row in validation_data.iterrows():
  ids.append(row['input_ids'])
  masks.append(row['attention_mask'])
  lbls.append(row['Label'])


validation_features = {'input_ids': ids, 
                  'attention_mask': masks,
                  'labels': lbls}



valds = Dataset.from_dict(validation_features)
print(valds)

ids = []
masks = []
lbls = []
for index, row in train_data.iterrows():
  ids.append(row['input_ids'])
  masks.append(row['attention_mask'])
  lbls.append(row['Label'])


train_features = {'input_ids': ids, 
                  'attention_mask': masks,
                  'labels': lbls}

                  
trainds = Dataset.from_dict(train_features)
print(trainds)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 340
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1360
})


In [24]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments

model = RobertaForSequenceClassification.from_pretrained('roberta-base',num_labels=5)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [25]:
args = TrainingArguments(
    output_dir='Trainer_0',
    evaluation_strategy='epoch',
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0,
)

In [26]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments

model = RobertaForSequenceClassification.from_pretrained('roberta-base',num_labels=5)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [27]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=trainds,
    eval_dataset=valds,
    tokenizer=tokenizer
)

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.172574


TrainOutput(global_step=85, training_loss=0.4164402681238511, metrics={'train_runtime': 136.4707, 'train_samples_per_second': 9.966, 'train_steps_per_second': 0.623, 'total_flos': 357840673751040.0, 'train_loss': 0.4164402681238511, 'epoch': 1.0})

In [29]:
trainer.evaluate()

{'eval_loss': 0.17257419228553772,
 'eval_runtime': 10.4053,
 'eval_samples_per_second': 32.676,
 'eval_steps_per_second': 2.114,
 'epoch': 1.0}

In [30]:
final_output = trainer.predict(valds)

In [31]:
plbl = np.argmax(final_output.predictions, axis=1)

In [32]:
from sklearn.metrics import classification_report
print(classification_report(plbl, validation_labels,digits=3))

              precision    recall  f1-score   support

           0      1.000     0.927     0.962        55
           1      0.932     0.944     0.938        72
           2      0.985     1.000     0.992        64
           3      0.976     0.988     0.982        83
           4      0.910     0.924     0.917        66

    accuracy                          0.959       340
   macro avg      0.961     0.957     0.958       340
weighted avg      0.959     0.959     0.959       340



**Hyperparameter-1**

In [33]:
args1 = TrainingArguments(
    output_dir='Trainer_1',
    evaluation_strategy='epoch',
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0,
)

In [34]:
trainer1 = Trainer(
    model=model,
    args=args1,
    train_dataset=trainds,
    eval_dataset=valds,
    tokenizer=tokenizer
)

In [35]:
trainer1.train()

Epoch,Training Loss,Validation Loss
1,No log,0.496036
2,No log,0.374873


TrainOutput(global_step=340, training_loss=0.5575911129222196, metrics={'train_runtime': 280.2675, 'train_samples_per_second': 9.705, 'train_steps_per_second': 1.213, 'total_flos': 715410006917664.0, 'train_loss': 0.5575911129222196, 'epoch': 2.0})

In [36]:
trainer1.evaluate()

{'eval_loss': 0.3748733103275299,
 'eval_runtime': 10.2137,
 'eval_samples_per_second': 33.289,
 'eval_steps_per_second': 4.21,
 'epoch': 2.0}

In [37]:
final_output1 = trainer1.predict(valds)

In [38]:
plbl1 = np.argmax(final_output1.predictions, axis=1)

In [39]:
from sklearn.metrics import classification_report
print(classification_report(plbl1, validation_labels,digits=3))

              precision    recall  f1-score   support

           0      0.980     0.893     0.935        56
           1      0.849     1.000     0.919        62
           2      0.954     0.969     0.961        64
           3      0.964     0.964     0.964        84
           4      0.955     0.865     0.908        74

    accuracy                          0.938       340
   macro avg      0.941     0.938     0.937       340
weighted avg      0.942     0.938     0.938       340



**Hyperparameter-2**

In [40]:
args2 = TrainingArguments(
    output_dir='Trainer_2',
    evaluation_strategy='epoch',
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0,
)

In [41]:
trainer2 = Trainer(
    model=model,
    args=args2,
    train_dataset=trainds,
    eval_dataset=valds,
    tokenizer=tokenizer
)

In [42]:
trainer2.train()

Epoch,Training Loss,Validation Loss
1,No log,1.609226


TrainOutput(global_step=340, training_loss=1.6408255184397977, metrics={'train_runtime': 150.2004, 'train_samples_per_second': 9.055, 'train_steps_per_second': 2.264, 'total_flos': 352362471800520.0, 'train_loss': 1.6408255184397977, 'epoch': 1.0})

In [43]:
trainer2.evaluate()

{'eval_loss': 1.6092256307601929,
 'eval_runtime': 10.253,
 'eval_samples_per_second': 33.161,
 'eval_steps_per_second': 8.29,
 'epoch': 1.0}

In [44]:
final_output2 = trainer2.predict(valds)

In [45]:
plbl2 = np.argmax(final_output2.predictions, axis=1)

In [46]:
from sklearn.metrics import classification_report
print(classification_report(plbl2, validation_labels,digits=3))

              precision    recall  f1-score   support

           0      0.000     0.000     0.000         0
           1      0.000     0.000     0.000         0
           2      0.000     0.000     0.000         0
           3      0.000     0.000     0.000         0
           4      1.000     0.197     0.329       340

    accuracy                          0.197       340
   macro avg      0.200     0.039     0.066       340
weighted avg      1.000     0.197     0.329       340



**Hyperparameter-3**

In [47]:
args3 = TrainingArguments(
    output_dir='Trainer_2',
    evaluation_strategy='epoch',
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0,
)

In [48]:
trainer3 = Trainer(
    model=model,
    args=args3,
    train_dataset=trainds,
    eval_dataset=valds,
    tokenizer=tokenizer
)

In [49]:
trainer3.train()

Epoch,Training Loss,Validation Loss
1,No log,1.615880


TrainOutput(global_step=340, training_loss=1.6379256304572611, metrics={'train_runtime': 150.124, 'train_samples_per_second': 9.059, 'train_steps_per_second': 2.265, 'total_flos': 352362471800520.0, 'train_loss': 1.6379256304572611, 'epoch': 1.0})

In [50]:
trainer3.evaluate()

{'eval_loss': 1.6158798933029175,
 'eval_runtime': 10.235,
 'eval_samples_per_second': 33.219,
 'eval_steps_per_second': 8.305,
 'epoch': 1.0}

In [51]:
final_output3 = trainer3.predict(valds)

In [52]:
plbl3 = np.argmax(final_output3.predictions, axis=1)

In [53]:
from sklearn.metrics import classification_report
print(classification_report(plbl3, validation_labels,digits=3))

              precision    recall  f1-score   support

           0      0.000     0.000     0.000         0
           1      0.000     0.000     0.000         0
           2      0.000     0.000     0.000         0
           3      0.000     0.000     0.000         0
           4      1.000     0.197     0.329       340

    accuracy                          0.197       340
   macro avg      0.200     0.039     0.066       340
weighted avg      1.000     0.197     0.329       340

